# 🌟 XL Lora Trainer

❗ 推薦使用 Colab Premium。理想情況下，您可以將運行時間變更為 A100 並使用最大批次大小。 但是，如果您載入diffusers，您仍然可以免費訓練，只是需要更長的時間。


This colab is based on the work of [Kohya-ss](https://github.com/kohya-ss/sd-scripts) and [Linaqruf](https://github.com/Linaqruf/kohya-trainer). Thank you!

### ⭕ 免責聲明
本文件的目的是研究機器學習推理領域的前沿技術。  
請閱讀並遵守 [Google Colab 指南](https://research.google.com/colaboratory/faq.html) 和其 [服務條款](https://research.google.com/colaboratory/tos_v3.html)。


| |GitHub|TrainerXL|Dataset Maker WDV2|Dataset Maker WDV3| |
|:--|:-:|:-:|:-:||:-:|
| 🏠 **Original Proyect** |[![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | |
|**Modified By WhiteZ**| [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/gwhitez/Lora-Trainer-XL) |   [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/gwhitez/Lora-Trainer-XL/blob/main/Fix_Lora_Trainer_XL.ipynb) |[![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/gwhitez/Lora-Trainer-XL/blob/main/Dataset_Maker%20By%20WhiteZ.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/gwhitez/Lora-Trainer-XL/blob/main/Waifu_Diffusion_V3_Dataser_Maker.ipynb)


In [ ]:
import os
import re
import toml
from time import time
from IPython.display import Markdown, display
import time

#@title ## 🚩 Start Here
#auto off by ZeroProtecPlus https://github.com/ZeroProtecPlus
#@markdown ### 💤 Auto Off

#@markdown 如果選中此選項，訓練完成後環境將自動斷開連接。
auto_off   = True  # @param {type: "boolean"}
if auto_off:
    print("\033[96mAuto Off encendido\033[0m")
if not auto_off:
    print("\033[96mAuto Off apagado\033[0m")

# These carry information from past executions
if "model_url" in globals():
  old_model_url = model_url
else:
  old_model_url = None
if "dependencies_installed" not in globals():
  dependencies_installed = False
if "model_file" not in globals():
  model_file = None

# These may be set by other cells, some are legacy
if "custom_dataset" not in globals():
  custom_dataset = None
if "override_dataset_config_file" not in globals():
  override_dataset_config_file = None
if "override_config_file" not in globals():
  override_config_file = None

COLAB = True
SOURCE = "https://github.com/qaneel/kohya-trainer"
COMMIT = None
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True
try:
  LOWRAM = int(next(line.split()[1] for line in open('/proc/meminfo') if "MemTotal" in line)) / (1024**2) < 15
except:
  LOWRAM = False



#@markdown ### ▶️ 設定
#@markdown 您的專案名稱將與包含圖片的資料夾名稱相同。不能包含空格，如果名稱過長，可以使用 `底線`。
project_name = "" #@param {type:"string"}
project_name = project_name.strip()
#@markdown 資料夾結構無關緊要，純粹是為了方便。請確保始終選擇相同的結構。我喜歡按專案進行整理。
folder_structure = "Organize by project (MyDrive/Loras/project_name/dataset)" #@param ["Organize by category (MyDrive/lora_training/datasets/project_name)", "Organize by project (MyDrive/Loras/project_name/dataset)"]
#@markdown 選擇要下載並用於訓練的模型。您也可以選擇自己的模型，粘貼其下載連結或從您的 Google Drive 中使用以 `/content/drive/MyDrive` 開頭的檔案。
training_model = "WAI-NSFW-illustrious" # @param ["Pony Diffusion V6 XL","Animagine XL V3","Illustrious","NoobAI-XL0.75","NoobAI-XL0.5","Stable Diffusion XL 1.0 base","Pony_Realism","WAI-NSFW-illustrious"]
optional_custom_training_model_url = "" #@param {type:"string"}
#@markdown 使用 diffusers 模型會消耗較少資源。上述選項在使用或不使用 diffusers 時都可以運行。
load_diffusers = True #@param {type:"boolean"}
#@markdown 如果您希望隨時間可視化訓練進度，請使用 wandb。
wandb_key = "" #@param {type:"string"}


if optional_custom_training_model_url:
  model_url = optional_custom_training_model_url
elif "Pony" in training_model:
  if load_diffusers:
    model_url = "https://huggingface.co/WhiteAiZ/Pony_diffusion_v6_diffusers_fp16"
  else:
    model_url = "https://huggingface.co/WhiteAiZ/PonyXL/resolve/main/PonyDiffusionV6XL.safetensors"
  model_file = "/content/ponyDiffusionV6XL.safetensors"
elif "Animagine" in training_model:
  if load_diffusers:
    model_url = "https://huggingface.co/cagliostrolab/animagine-xl-3.0"
  else:
    model_url = "https://civitai.com/api/download/models/293564"
  model_file = "/content/animagineXLV3.safetensors"
elif "Illustrious" in training_model:
  if load_diffusers:
    model_url = "https://huggingface.co/OnomaAIResearch/Illustrious-xl-early-release-v0"
  else:
    model_url = "https://huggingface.co/OnomaAIResearch/Illustrious-xl-early-release-v0/resolve/main/Illustrious-XL-v0.1.safetensors"
elif "WAI-NSFW-illustrious" in training_model:
  if load_diffusers:
    model_url = "https://civitai.com/api/download/models/1183765?type=Model&format=SafeTensor&size=pruned&fp=fp16"
  else:
    model_url = "https://civitai.com/api/download/models/1183765?type=Model&format=SafeTensor&size=pruned&fp=fp16"
elif "NoobAI-XL0.75" in training_model:
  if load_diffusers:
    model_url = "https://huggingface.co/Laxhar/noobai-XL-0.75"
  else:
    model_url = "https://huggingface.co/Laxhar/noobai-XL-0.75/resolve/main/NoobAI-XL-v0.75.safetensors"
elif "NoobAI-XL0.5" in training_model:
  if load_diffusers:
    model_url = "https://huggingface.co/Laxhar/noobai-XL-0.5"
  else:
    model_url = "https://huggingface.co/Laxhar/noobai-XL-0.5/resolve/main/NoobAI-XL-v0.5.safetensors"
else:
  if load_diffusers:
    model_url = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/"
  else:
    model_url = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors"

  if load_diffusers:
    model_url = "https://huggingface.co/stablediffusionapi/pony-realism"
  else:
    model_url = "https://huggingface.co/LyliaEngine/ponyRealism_v21MainVAE/resolve/main/ponyRealism_v21MainVAE.safetensors"
  model_file = "/content/ponyrealism.safetensors"

if load_diffusers:
  vae_file= "stabilityai/sdxl-vae"
else:
  vae_url = "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors"
  vae_file = "/content/sdxl_vae.safetensors"


#@markdown ### ▶️ 處理
#@markdown 預設角色的解析度為 1024。其他可選解析度包括 896（建議用於角色，或保持 1024）以及 768（建議用於風格，該解析度可使用更多的重複次數）。
resolution = 1024 #@param {type:"number", min:768, max:1536, step:128}
#@markdown 如果您的數據集較小，啟用 `Flip Aug` 會很有幫助，適用於等距角色。它會將所有圖片進行鏡像翻轉學習，從而加倍學習數據，但可能會影響帶有紋身、標記、疤痕等的角色。
flip_aug = False #@param {type:"boolean"}
caption_extension = ".txt" # @param [".txt",".caption"]
#@markdown 混合動畫標籤有助於改進學習和提示輸入。啟用標籤時，激活標籤將被添加到每個文本檔案的開頭，並且不會被混合。
shuffle_tags = True #@param {type:"boolean"}
shuffle_caption = shuffle_tags
activation_tags = "3" #@param [0,1,2,3]
keep_tokens = int(activation_tags)

#@markdown ### ▶️ 步驟 <p>
#@markdown 您的圖片在訓練過程中將重複此次數。我建議圖片的數量乘以重複次數在 200 到 400 之間。
num_repeats = 50 #@param {type:"number"}
#@markdown 選擇您想要訓練的時長。一個不錯的起點是大約 10 個 epoch 或大約 2000 個步驟。<p>
#@markdown 一個 epoch 的步驟數等於：圖片數量乘以重複次數，再除以批量大小。<p>
preferred_unit = "Epochs" #@param ["Epochs", "Steps"]
how_many = 10 #@param {type:"number"}
max_train_epochs = how_many if preferred_unit == "Epochs" else None
max_train_steps = how_many if preferred_unit == "Steps" else None
#@markdown 保存更多的 epoch 可以更好地比較您的 LoRA 進度。
save_every_n_epochs = 1 #@param {type:"number"}
keep_only_last_n_epochs = 10 #@param {type:"number"}
if not save_every_n_epochs:
  save_every_n_epochs = max_train_epochs
if not keep_only_last_n_epochs:
  keep_only_last_n_epochs = max_train_epochs

#@markdown ### ▶️ 學習
#@markdown 學習率對結果非常重要。如果您有大量圖片，或者 dim 和 alpha 設定較高，請將 unet 的學習率設定為 2e-4 或更低。<p>
#@markdown 文本編碼器有助於讓 LoRA 更好地學習概念。建議將其設為 unet 的一半或五分之一。如果您正在訓練風格，也可以設為 0。
unet_lr = 1e-4 #@param {type:"number"}
text_encoder_lr = 1e-5 #@param {type:"number"}
#@markdown 調度器是指導學習率的算法。如果不確定，選擇 "constant"，忽略數值。個人推薦使用 `cosine_with_restarts` 並設置 3 次重啟。
lr_scheduler = "cosine_with_restarts" #@param ["constant", "cosine", "cosine_with_restarts", "constant_with_warmup", "linear", "polynomial"]
lr_scheduler_number = 3 #@param {type:"number"}
#@markdown 訓練期間，用於「預熱」學習率的步驟數以提高效率。建議設為 5%。
lr_warmup_ratio = 0.05 #@param {type:"slider", min:0.0, max:0.2, step:0.01}
lr_warmup_steps = 0
#@markdown 調整損失隨時間的變化，使學習更加高效。文檔建議設為 5.0，我建議動畫風格設為 8.0。更高的值會降低嚴格性，設為 0 可禁用。
min_snr_gamma = 8.0 #@param {type:"slider", min:4.0, max:16.0, step:0.5}
#@markdown MultiNoise 可以幫助改善顏色平衡（更深的黑色、更亮的白色）。但可能會影響眼睛的生成，僅在必要時使用。
multinoise = True #@param {type:"boolean"}

#@markdown ### ▶️ 結構
#@markdown LoRA 是經典類型，適用於多種用途。LoCon 更適合藝術風格，因為它有更多的層來學習數據集的更多細節。
lora_type = "LoRA" #@param ["LoRA", "LoCon"]

#@markdown 以下是針對 XL 設定的一些推薦值：

#@markdown | 類型 | network_dim | network_alpha | conv_dim | conv_alpha |
#@markdown | :---: | :---: | :---: | :---: | :---: |
#@markdown | 角色 LoRA | 4 | 16 |   |   |
#@markdown | 通用與風格 LoRA | 8 | 4 |   |   |
#@markdown | 風格 LoCon | 16 | 8 | 16 | 8 |

#@markdown 更高的 dim 意味著更大的 LoRA，可以容納更多信息，但更高並不總是更好。
network_dim = 32 #@param {type:"number", min:1, max:32, step:1}
network_alpha = 16 #@param {type:"number", min:1, max:32, step:1}
#@markdown 以下兩個值僅適用於 LoCon 的附加層。
conv_dim = 16 #@param {type:"number", min:1, max:32, step:1}
conv_alpha = 8 #@param {type:"number", min:1, max:32, step:1}


network_module = "networks.lora"
network_args = None
if lora_type.lower() == "locon":
  network_args = [f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}"]

#@markdown ### ▶️ 訓練
#@markdown 根據您的 Colab 配置調整以下參數。
#@markdown 如果您使用的是免費等級，請在本單元格頂部選擇一個 diffusers 模型。
#@markdown
#@markdown 批量大小（batch size）越大，訓練速度通常越快，但需要更多的記憶體。
train_batch_size = 16 #@param {type:"slider", min:1, max:16, step:1}
#@markdown 我尚未發現 sdpa 和 xformers 之間有實質差異。
cross_attention = "sdpa" #@param ["sdpa", "xformers"]
#@markdown 如果您使用的是 A100 GPU，請啟用 bf16（僅適用於 Colab Pro）。
mixed_precision = "fp16" #@param ["bf16", "fp16"]
#@markdown 在 Google Drive 中緩存潛變數（Latent Caching）會在每張圖片旁生成一個 250 KB 的檔案，但能顯著降低記憶體使用量。
cache_latents = True #@param {type:"boolean"}
cache_latents_to_drive = False #@param {type:"boolean"}
#@markdown 以下選項會禁用 shuffle_tags 並停用文本編碼器的訓練。
cache_text_encoder_outputs = False  # @param {type:"boolean"}

#@markdown ### ▶️ 高級設置
#@markdown 優化器是用於訓練的算法。默認的 AdaFactor 表現良好，而 Prodigy 自動管理學習率，具備多種優勢，例如需要較少的步驟即可完成訓練，對小型數據集效果更佳。
optimizer = "Prodigy" #@param ["AdamW8bit", "Prodigy", "DAdaptation", "DadaptAdam", "DadaptLion", "AdamW", "Lion", "SGDNesterov", "SGDNesterov8bit", "AdaFactor"]
#@markdown AdamW8bit 的推薦參數：`weight_decay=0.1 betas=[0.9,0.99]`<p>
#@markdown Prodigy 的推薦參數：`decouple=True weight_decay=0.01 betas=[0.9,0.999] d_coef=2 use_bias_correction=True safeguard_warmup=True`<p>
#@markdown AdaFactor 的推薦參數：`scale_parameter=False relative_step=False warmup_init=False`
#markdown CAME 的推薦參數：`weight_decay=0.04`
optimizer_args = "decouple=True weight_decay=0.01 betas=[0.9,0.999] d_coef=2 use_bias_correction=True safeguard_warmup=True" #@param {type:"string"}
optimizer_args = [a.strip() for a in optimizer_args.split(' ') if a]
#@markdown 如果選擇 Dadapt 或 Prodigy 並勾選推薦設置，以下推薦值將覆蓋所有之前的設置：<p>
#markdown `unet_lr=0.75`, `text_encoder_lr=0.75`, `network_alpha=network_dim`<p>
#markdown 如果選擇 CAME + REX 並勾選推薦設置，以下推薦值將覆蓋所有之前的設置：<p>
#markdown `unet_lr=1e-4`, `text_encoder_lr=1e-6, betas=default_came_betas`
recommended_values = True #@param {type:"boolean"}

if any(opt in optimizer.lower() for opt in ["dadapt", "prodigy"]):
  if recommended_values:
    unet_lr = 0.75
    text_encoder_lr = 0.75
    network_alpha = network_dim
elif "CAME" in optimizer:
  optimizer = "CAME"
  lr_scheduler = "REX"
  if recommended_values:
    unet_lr = 1e-4
    text_encoder_lr = 1e-6
    for i in range(len(optimizer_args)):
      if "betas" in optimizer_args[i]:
        optimizer_args.pop(i)
        break

lr_scheduler_num_cycles = lr_scheduler_number if lr_scheduler == "cosine_with_restarts" else 0
lr_scheduler_power = lr_scheduler_number if lr_scheduler == "polynomial" else 0


#@markdown ### ▶️ 準備
#@markdown 現在您可以運行此單元格來訓練您的 Lora。祝你好運！ <p>


# 👩‍💻 Cool code goes here

root_dir = "/content" if COLAB else pathlib.Path.home() / "Loras"
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "kohya-trainer")

if "/Loras" in folder_structure:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
  log_folder    = os.path.join(main_dir, "_logs")
  config_folder = os.path.join(main_dir, project_name)
  images_folder = os.path.join(main_dir, project_name, "dataset")
  output_folder = os.path.join(main_dir, project_name, "output")
else:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
  images_folder = os.path.join(main_dir, "datasets", project_name)
  output_folder = os.path.join(main_dir, "output", project_name)
  config_folder = os.path.join(main_dir, "config", project_name)
  log_folder    = os.path.join(main_dir, "log")

config_file = os.path.join(config_folder, "training_config.toml")
dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")

def install_dependencies():
  os.chdir(root_dir)
  !git clone {SOURCE} {repo_dir}
  os.chdir(repo_dir)
  if COMMIT:
    !git reset --hard {COMMIT}
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/train_network_xl_wrapper.py -q -O train_network_xl_wrapper.py
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/dracula.py -q -O dracula.py

  !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4

  !apt -y update -qq
  !apt -y install aria2 -qq
  !pip install torch==2.5.1+cu121 accelerate==0.19.0 transformers==4.30.2 diffusers==0.18.2 -q \
    bitsandbytes==0.40.0.post4 flax==0.7.5 opencv-python jax==0.4.23 jaxlib==0.4.23 -q \
    pytorch-lightning==1.9.0 voluptuous==0.13.1 toml==0.10.2 ftfy==6.1.1 einops==0.6.0 -q \
    safetensors pygments huggingface-hub==0.22.0 wandb invisible-watermark==0.2.0 open-clip-torch==2.20.0 -q \
    dadaptation==3.1 prodigyopt==1.0 lion-pytorch==0.1.2 -q
  !pip install -e .
  if cross_attention == "xformers":
    !pip install -q xformers
  if "CAME" in optimizer:
    !pip install came-pytorch
    !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/train_util.py -q -O library/train_util.py

  # patch kohya for minor stuff
  if LOWRAM:
    !sed -i "s@cpu@cuda@" library/model_util.py
  if LOAD_TRUNCATED_IMAGES:
    !sed -i 's/from PIL import Image/from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True/g' library/train_util.py # fix truncated jpegs error
  if BETTER_EPOCH_NAMES:
    !sed -i 's/{:06d}/{:02d}/g' library/train_util.py # make epoch names shorter
    !sed -i 's/"." + args.save_model_as)/"-{:02d}.".format(num_train_epochs) + args.save_model_as)/g' train_network.py # name of the last epoch will match the rest

  from accelerate.utils import write_basic_config
  if not os.path.exists(accelerate_config_file):
    write_basic_config(save_location=accelerate_config_file)

  os.environ["LD_PRELOAD"] = "/content/libtcmalloc_minimal.so.4"
  os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
  os.environ["BITSANDBYTES_NOWELCOME"] = "1"
  os.environ["SAFETENSORS_FAST_GPU"] = "1"
  os.environ["PYTHONWARNINGS"] = "ignore"

def validate_dataset():
  global lr_warmup_steps, lr_warmup_ratio, caption_extension, keep_tokens, model_url
  supported_types = (".png", ".jpg", ".jpeg", ".webp", ".bmp")

  if model_url.startswith("/content/drive/") and not os.path.exists(model_url):
    print("💥 Error: The custom training model you specified was not found in your Google Drive.")
    return

  print("\n💿 Checking dataset...")
  if not project_name.strip() or any(c in project_name for c in " .()\"'\\/"):
    print("💥 Error: Please choose a valid project name.")
    return

  # Find the folders and files
  if custom_dataset:
    try:
      datconf = toml.loads(custom_dataset)
      datasets = [d for d in datconf["datasets"][0]["subsets"]]
    except:
      print(f"💥 Error: Your custom dataset is invalid or contains an error! Please check the original template.")
      return
    reg = [d.get("image_dir") for d in datasets if d.get("is_reg", False)]
    datasets_dict = {d["image_dir"]: d["num_repeats"] for d in datasets}
    folders = datasets_dict.keys()
    files = [f for folder in folders for f in os.listdir(folder)]
    images_repeats = {folder: (len([f for f in os.listdir(folder) if f.lower().endswith(supported_types)]), datasets_dict[folder]) for folder in folders}
  else:
    reg = []
    folders = [images_folder]
    files = os.listdir(images_folder)
    images_repeats = {images_folder: (len([f for f in files if f.lower().endswith(supported_types)]), num_repeats)}

  # Validation
  for folder in folders:
    if not os.path.exists(folder):
      print(f"💥 Error: The folder {folder.replace('/content/drive/', '')} doesn't exist.")
      return
  for folder, (img, rep) in images_repeats.items():
    if not img:
      print(f"💥 Error: Your {folder.replace('/content/drive/', '')} folder is empty.")
      return
  test_files = []
  for f in files:
    if not f.lower().endswith((caption_extension, ".npz")) and not f.lower().endswith(supported_types):
      print(f"💥 Error: Invalid file in dataset: \"{f}\". Aborting.")
      return
    for ff in test_files:
      if f.endswith(supported_types) and ff.endswith(supported_types) \
          and os.path.splitext(f)[0] == os.path.splitext(ff)[0]:
        print(f"💥 Error: The files {f} and {ff} cannot have the same name. Aborting.")
        return
    test_files.append(f)

  if caption_extension and not [txt for txt in files if txt.lower().endswith(caption_extension)]:
    caption_extension = ""

  # Pretty stuff

  pre_steps_per_epoch = sum(img*rep for (img, rep) in images_repeats.values())
  steps_per_epoch = pre_steps_per_epoch/train_batch_size
  total_steps = max_train_steps or int(max_train_epochs*steps_per_epoch)
  estimated_epochs = int(total_steps/steps_per_epoch)
  lr_warmup_steps = int(total_steps*lr_warmup_ratio)

  for folder, (img, rep) in images_repeats.items():
    print("📁"+folder.replace("/content/drive/", "") + (" (Regularization)" if folder in reg else ""))
    print(f"📈 Found {img} images with {rep} repeats, equaling {img*rep} steps.")
  print(f"📉 Divide {pre_steps_per_epoch} steps by {train_batch_size} batch size to get {steps_per_epoch} steps per epoch.")
  if max_train_epochs:
    print(f"🔮 There will be {max_train_epochs} epochs, for around {total_steps} total training steps.")
  else:
    print(f"🔮 There will be {total_steps} steps, divided into {estimated_epochs} epochs and then some.")

  if total_steps > 10000:
    print("💥 Error: Your total steps are too high. You probably made a mistake. Aborting...")
    return

  return True

def create_config():
  global dataset_config_file, config_file, model_file

  if override_config_file:
    config_file = override_config_file
    print(f"\n⭕ Using custom config file {config_file}")
  else:
    config_dict = {
      "network_arguments": {
        "unet_lr": unet_lr,
        "text_encoder_lr": text_encoder_lr if not cache_text_encoder_outputs else 0,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_module": network_module,
        "network_args": network_args,
        "network_train_unet_only": text_encoder_lr == 0 or cache_text_encoder_outputs,
      },
      "optimizer_arguments": {
        "learning_rate": unet_lr,
        "lr_scheduler": lr_scheduler,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
        "lr_warmup_steps": lr_warmup_steps if lr_scheduler != "constant" else None,
        "optimizer_type": optimizer,
        "optimizer_args": optimizer_args if optimizer_args else None,
      },
      "training_arguments": {
        "pretrained_model_name_or_path": model_file,
        "vae": vae_file,
        "max_train_steps": max_train_steps,
        "max_train_epochs": max_train_epochs,
        "train_batch_size": train_batch_size,
        "seed": 42,
        "max_token_length": 225,
        "xformers": cross_attention == "xformers",
        "sdpa": cross_attention == "sdpa",
        "min_snr_gamma": min_snr_gamma if min_snr_gamma > 0 else None,
        "lowram": LOWRAM,
        "no_half_vae": True,
        "gradient_checkpointing": True,
        "gradient_accumulation_steps": 1,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "mixed_precision": mixed_precision,
        "full_bf16": mixed_precision == "bf16",
        "cache_latents": cache_latents,
        "cache_latents_to_disk": cache_latents_to_drive,
        "cache_text_encoder_outputs": cache_text_encoder_outputs,
        "min_timestep": 0,
        "max_timestep": 1000,
        "prior_loss_weight": 1.0,
        "multires_noise_iterations": 6 if multinoise else None,
        "multires_noise_discount": 0.3 if multinoise else None,
      },
      "saving_arguments": {
        "save_precision": "fp16",
        "save_model_as": "safetensors",
        "save_every_n_epochs": save_every_n_epochs,
        "save_last_n_epochs": keep_only_last_n_epochs,
        "output_name": project_name,
        "output_dir": output_folder,
        "log_prefix": project_name,
        "logging_dir": log_folder,
        "wandb_api_key": wandb_key if wandb_key else None,
        "log_with": "wandb" if wandb_key else None,
      }
    }

    for key in config_dict:
      if isinstance(config_dict[key], dict):
        config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

    with open(config_file, "w") as f:
      f.write(toml.dumps(config_dict))
    print(f"\n📄 Config saved to {config_file}")

  if override_dataset_config_file:
    dataset_config_file = override_dataset_config_file
    print(f"⭕ Using custom dataset config file {dataset_config_file}")
  else:
    dataset_config_dict = {
      "general": {
        "resolution": resolution,
        "shuffle_caption": shuffle_caption and not cache_text_encoder_outputs,
        "keep_tokens": keep_tokens,
        "flip_aug": flip_aug,
        "caption_extension": caption_extension,
        "enable_bucket": True,
        "bucket_no_upscale": False,
        "bucket_reso_steps": 64,
        "min_bucket_reso": 256,
        "max_bucket_reso": 4096,
      },
      "datasets": toml.loads(custom_dataset)["datasets"] if custom_dataset else [
        {
          "subsets": [
            {
              "num_repeats": num_repeats,
              "image_dir": images_folder,
              "class_tokens": None if caption_extension else project_name
            }
          ]
        }
      ]
    }

    for key in dataset_config_dict:
      if isinstance(dataset_config_dict[key], dict):
        dataset_config_dict[key] = {k: v for k, v in dataset_config_dict[key].items() if v is not None}

    with open(dataset_config_file, "w") as f:
      f.write(toml.dumps(dataset_config_dict))
    print(f"📄 Dataset config saved to {dataset_config_file}")

def download_model():
  global old_model_url, model_url, model_file, vae_url, vae_file
  real_model_url = model_url  # There was a reason for having a separate variable but I forgot what it was.

  if real_model_url.startswith("/content/drive/"):
    # Local model, already checked to exist
    model_file = real_model_url
    print(f"📁 Using local model file: {model_file}")
    # Validation
    if model_file.lower().endswith(".safetensors"):
      from safetensors.torch import load_file as load_safetensors
      try:
        test = load_safetensors(model_file)
        del test
      except:
        return False
    elif model_file.lower().endswith(".ckpt"):
      from torch import load as load_ckpt
      try:
        test = load_ckpt(model_file)
        del test
      except:
        return False
    return True

  else:
    # Downloadable model
    if load_diffusers:
      if 'huggingface.co' in real_model_url:
          match = re.search(r'huggingface\.co/([^/]+)/([^/]+)', real_model_url)
          if match:
              username = match.group(1)
              model_name = match.group(2)
              model_file = f"{username}/{model_name}"
              from huggingface_hub import HfFileSystem
              fs = HfFileSystem()
              existing_folders = set(fs.ls(model_file, detail=False))
              necessary_folders = [ "scheduler", "text_encoder", "text_encoder_2", "tokenizer", "tokenizer_2", "unet", "vae" ]
              if all(f"{model_file}/{folder}" in existing_folders for folder in necessary_folders):
                print("🍃 Diffusers model identified.")  # Will be handled by kohya
                return True
      raise ValueError("💥 Failed to load Diffusers model. If this model is not Diffusers, have you tried turning it off at the top of the colab?")

    # Define local filename
    if not model_file:
      if real_model_url.lower().endswith((".ckpt", ".safetensors")):
        model_file = f"/content{real_model_url[real_model_url.rfind('/'):]}"
      else:
        model_file = "/content/downloaded_model.safetensors"
        if os.path.exists(model_file):
          !rm "{model_file}"

    # HuggingFace
    if m := re.search(r"(?:https?://)?(?:www\.)?huggingface\.co/[^/]+/[^/]+/blob", real_model_url):
      real_model_url = real_model_url.replace("blob", "resolve")
    # Civitai
    elif m := re.search(r"(?:https?://)?(?:www\\.)?civitai\.com/models/([0-9]+)(/[A-Za-z0-9-_]+)?", real_model_url):
      if m.group(2):
        model_file = f"/content{m.group(2)}.safetensors"
      if m := re.search(r"modelVersionId=([0-9]+)", real_model_url):
        real_model_url = f"https://civitai.com/api/download/models/{m.group(1)}"
      else:
        raise ValueError("💥 optional_custom_training_model contains a civitai link, but the link doesn't include a modelVersionId. You can also right click the download button to copy the direct download link.")

    # Download checkpoint
    !aria2c "{real_model_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{model_file}"

    # Download VAE
    if not os.path.exists(vae_file):
      !aria2c "{vae_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{vae_file}"

    # Validation

    if model_file.lower().endswith(".safetensors"):
      from safetensors.torch import load_file as load_safetensors
      try:
        test = load_safetensors(model_file)
        del test
      except:
        new_model_file = os.path.splitext(model_file)[0]+".ckpt"
        !mv "{model_file}" "{new_model_file}"
        model_file = new_model_file
        print(f"Renamed model to {os.path.splitext(model_file)[0]}.ckpt")

    if model_file.lower().endswith(".ckpt"):
      from torch import load as load_ckpt
      try:
        test = load_ckpt(model_file)
        del test
      except:
        return False

  return True

def main():
  global dependencies_installed

  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  for dir in (main_dir, deps_dir, repo_dir, log_folder, images_folder, output_folder, config_folder):
    os.makedirs(dir, exist_ok=True)

  if not validate_dataset():
      return

  if not dependencies_installed:
    print("\n🏭 Installing dependencies...\n")
    t0 = time.time()
    install_dependencies()
    t1 = time.time()
    dependencies_installed = True
    print(f"\n✅ Installation finished in {int(t1-t0)} seconds.")
  else:
    print("\n✅ Dependencies already installed.")

  if old_model_url != model_url or not model_file or not os.path.exists(model_file):
    print("\n🔄 Getting model...")
    if not download_model():
      print("\n💥 Error: The model you specified is invalid or corrupted."
            "\nIf you're using an URL, please check that the model is accessible without being logged in."
            "\nYou can try civitai or huggingface URLs, or a path in your Google Drive starting with /content/drive/MyDrive")
      return
    print()
  else:
    print("\n🔄 Model already downloaded.\n")

  create_config()

  print("\n⭐ Starting trainer...\n")
  os.chdir(repo_dir)

  !accelerate launch --quiet --config_file={accelerate_config_file} --num_cpu_threads_per_process=1 train_network_xl_wrapper.py --dataset_config={dataset_config_file} --config_file={config_file}

  if not get_ipython().__dict__.get('user_ns', {}).get('_exit_code', False):
    display(Markdown("### ✅ Done! [Go download your Lora from Google Drive](https://drive.google.com/drive/my-drive)\n"
                       "### There will be several files, you should try the latest version (the file with the largest number next to it)"))

main()

#auto off
if auto_off:
    print("\033[96mAuto Off encendido, el entorno se desconectara en 30 segundos\033[0m")
    from google.colab import runtime
    time.sleep(30)
    runtime.unassign()
else:
    print("\033[96mAuto off está desactivado, el entorno no se desconectará\033[0m")




## *️⃣ 額外功能  
您可以在開始訓練之前執行此操作。


In [ ]:
#@markdown 連接到 Google Drive 📁。以下的一些選項需要連接到 Google Drive，
#@markdown 請在此處連接。
from google.colab import drive
drive.mount('/content/drive')
print("連接到 Drive 成功！")

### 📚 多個資料夾在同一數據集中  
以下是一個範本，允許您在數據集中定義多個資料夾。您需要包括每個資料夾的位置，並可以為每個資料夾設置不同的重複次數。要添加更多資料夾，只需複製並粘貼以 `[[datasets.subsets]]` 開頭的部分。

啟用此功能後，主單元格中設定的重複次數以及通過專案名稱設定的主資料夾將被忽略。

您可以將其中一個設為正規化資料夾，只需添加 `is_reg = true`  
還可以為每個資料夾單獨設置不同的 `keep_tokens`、`flip_aug` 等參數。


In [ ]:
custom_dataset = """
[[datasets]]

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/KashiwazakiKana_IXL/dataset/21 25"
num_repeats = 25

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/KashiwazakiKana_IXL/dataset/CAUSAL 12 45"
num_repeats = 45

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/KashiwazakiKana_IXL/dataset/WORK 17 31"
num_repeats = 31

"""

In [ ]:
custom_dataset = None

In [ ]:
#@markdown ### 🔢 計算數據集
#@markdown 由於 Google Drive 無法直接計算資料夾中的文件數量，因此此功能將顯示所有資料夾及其子資料夾中的文件總數。

folder = "/content/drive/MyDrive/Loras/ejemplo/dataset" #@param {type:"string"}

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 Connecting to Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
exclude = ("_logs", "/output")
for i, (root, dirs, files) in enumerate(os.walk(folder, topdown=True)):
  dirs[:] = [d for d in dirs if all(ex not in d for ex in exclude)]
  images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.lower().endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images else f"{images:>4} images | {captions:>4} captions |"
  if tree[path] and others:
    tree[path] += f" {others:>4} other files"

pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))

In [ ]:
#@markdown ## 重複計算器 ⌛📝
#@markdown 計算用於訓練 Lora 的重複次數。請記住，在 `SDXL 和 Pony` 中，批量大小（batch size）為 `4`。
#@markdown 如果您使用的是 Colab Pro，請以 `8` 的批量大小計算您的重複次數。

# 定義變數
# 圖片數量
num_images = 40 # @param{type:"number"}
# 重複次數
num_repeats = 4 # @param{type:"number"}
# epoch 數量
num_epochs = 10 # @param{type:"number"}
# 批量大小
batch_size = 4 # @param{type:"number"}

# 計算結果
resultado = (num_images * num_repeats * num_epochs) / batch_size

# 顯示結果
print("\33[96m總重複次數為:\033[0m", resultado)



In [ ]:
#@markdown ## HuggingFace 🤗
from huggingface_hub import login
from huggingface_hub import HfApi
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError

# @markdown 創建一個 HuggingFace 帳戶 [點擊這裡](https://huggingface.co/)，如果您已有帳戶，直接使用您的 token 👇
# @markdown > 獲取您的 HuggingFace **寫入權限** token [點擊這裡](https://huggingface.co/settings/tokens) 👈
write_token = ""  # @param {type:"string"}
#@markdown 如果您希望將文件上傳到您的組織，請填寫此處，否則保持空白即可。
orgs_name = ""  # @param{type:"string"}
# @markdown 如果您的模型/數據集存儲庫不存在，將自動創建。
#@markdown 請勿使用空格，對於長名稱請使用 `底線`。

#@markdown `model_name` 用於上傳單個文件。
model_name = ""  # @param{type:"string"}
#@markdown `dataset_name` 用於上傳整個文件夾。
dataset_name = "loras"  # @param{type:"string"}
#@markdown 勾選此框以將您的存儲庫/數據集設置為私有，否則它將是公開的。
make_private = True  # @param{type:"boolean"}


def authenticate(write_token):
    login(write_token, add_to_git_credential=True)
    api = HfApi()
    return api.whoami(write_token), api


def create_repo(api, user, orgs_name, repo_name, repo_type, make_private=False):
    global model_repo
    global datasets_repo

    if orgs_name == "":
        repo_id = user["name"] + "/" + repo_name.strip()
    else:
        repo_id = orgs_name + "/" + repo_name.strip()

    try:
        validate_repo_id(repo_id)
        api.create_repo(repo_id=repo_id, repo_type=repo_type, private=make_private)
        print(f"{repo_type.capitalize()} repo '{repo_id}' didn't exist, creating repo")
    except HfHubHTTPError as e:
        print(f"{repo_type.capitalize()} repo '{repo_id}' exists, skipping create repo")

    if repo_type == "model":
        model_repo = repo_id
        print(f"{repo_type.capitalize()} repo '{repo_id}' link: https://huggingface.co/{repo_id}\n")
    else:
        datasets_repo = repo_id
        print(f"{repo_type.capitalize()} repo '{repo_id}' link: https://huggingface.co/datasets/{repo_id}\n")

user, api = authenticate(write_token)

if model_name:
    create_repo(api, user, orgs_name, model_name, "model", make_private)
if dataset_name:
    create_repo(api, user, orgs_name, dataset_name, "dataset", make_private)

In [ ]:
#@markdown ## 通過 huggingface_hub 上傳 (Lora) 🤗

from huggingface_hub import HfApi
from pathlib import Path

api = HfApi()
file_path = "/content/drive/MyDrive/Loras/Mi_proyecto/output/Mi_lora.safetensors" #@param {type :"string"}
#@markdown 評論（可選），可用於標識 Lora 的版本，例如：`pony, animagine, V1, V2 等...`

commit_message = ""  #@param {type :"string"}

if file_path != "":
  path_obj = Path(file_path)
  trained_model = path_obj.parts[-1]

  print(f"Uploading {trained_model} to https://huggingface.co/"+model_repo)
  print(f"Please wait...")

  api.upload_file(
      path_or_fileobj=file_path,
      path_in_repo=trained_model,
      repo_id=model_repo,
      commit_message=commit_message,
  )

  print(f"Upload success, located at https://huggingface.co/"+model_repo+"/blob/main/"+trained_model+"\n")
print('¡¡Subida finalizada!!')

In [ ]:
# @markdown ## 上傳資料夾 📁 至 HuggingFace 🤗
from huggingface_hub import HfApi
from pathlib import Path
import os

api = HfApi()

# @markdown 您可以將整個資料夾上傳到 HuggingFace，包括訓練好的 Loras，甚至可以上傳整個專案資料夾（包括子資料夾）。請耐心等待，上傳多個文件可能需要一些時間。
folder_path = "/content/drive/MyDrive/Loras/Mi_lora/output"  # @param {type :"string"}

# @markdown 上傳資料夾的自訂名稱
folder_name = "Mi_lora"  # @param {type :"string"}

# @markdown 評論（可選）
commit_message = ""  # @param {type :"string"}


if not commit_message:
    commit_message = "feat: upload folder"

def upload_folder(folder_path, folder_name):
    print(f"Uploading {folder_name} to https://huggingface.co/datasets/{datasets_repo}")
    print("Please wait...")

    api.upload_folder(
        folder_path=folder_path,
        path_in_repo=folder_name,
        repo_id=datasets_repo,
        repo_type="dataset",
        commit_message=commit_message,
        ignore_patterns=".ipynb_checkpoints",
    )
    print(f"Carga terminada, puedes encontrarlo en https://huggingface.co/datasets/{datasets_repo}/tree/main/{folder_name}\n")

def upload():
    if folder_path:
        upload_folder(folder_path, folder_name)

upload()


In [ ]:
#@markdown ### 📂 解壓數據集
#@markdown 在 Google Drive 上傳單個文件的速度較慢，因此如果您在電腦上有數據集，可以將其壓縮為 ZIP 文件並上傳。在這裡，您可以解壓縮它。
zip = "/content/drive/MyDrive/ABC.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/ABC/dataset" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ Done")

📂 Connecting to Google Drive...
Mounted at /content/drive
✅ Done


# 📈 繪製訓練結果  
在運行訓練器後可以執行此操作。如果您不熟悉此功能，則無需使用。  
第一個單元可能無法加載所有記錄。請繼續嘗試第二個單元，直到所有數據都被加載。


In [ ]:
%load_ext tensorboard
%tensorboard --logdir={log_folder}/

In [ ]:
from tensorboard import notebook
notebook.display(port=6006, height=800)